In [ ]:
# Import necessary libraries
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge, LinearRegression

from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor


import numpy as np
import pandas as pd

RSEED = 42

In [7]:
# 1. Load dataset
df = pd.read_csv('data_new/preprocessed_train_data_with_date_hol_concat.csv')
df.columns
df.isna().sum()

departure_point    0
arrival_point      0
departure_time     0
arrival_time       0
flight_status      0
aircraft_code      0
target             0
duration           0
dep_hour           0
dep_day            0
dep_month          0
dep_dayofweek      0
dep_quarter        0
dep_season         0
dep_is_weekend     0
dep_time_of_day    0
arr_hour           0
arr_day            0
arr_month          0
arr_dayofweek      0
arr_quarter        0
arr_season         0
arr_is_weekend     0
arr_time_of_day    0
route              0
departure_date     0
arrival_date       0
dep_temp           0
dep_precip         0
dep_wind           0
arr_temp           0
arr_precip         0
arr_wind           0
Country            0
City               0
is_holiday         0
holiday_length     0
dtype: int64

In [9]:
df

,departure_point,arrival_point,departure_time,arrival_time,flight_status,aircraft_code,target,duration,dep_hour,dep_day,...,dep_temp,dep_precip,dep_wind,arr_temp,arr_precip,arr_wind,Country,City,is_holiday,holiday_length
0,TUN,IST,2016-01-16 04:10:00,2016-01-16 06:45:00,ATA,TU 32AIMN,-19.0,9300.0,4,16,...,9.4,9.9,23.6,12.5,0.0,16.5,Tunisia,Tunis,0,0
1,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17:00:00,ATA,TU 736IOK,-48.0,10200.0,14,17,...,11.7,0.0,41.2,2.7,2.6,8.8,Tunisia,Djerba,0,0
2,TUN,MED,2016-01-20 19:40:00,2016-01-21 00:00:00,ATA,TU 320IMR,-16.0,15600.0,19,20,...,11.1,1.3,6.8,22.3,23.9,11.8,Tunisia,Tunis,0,0
3,IST,TUN,2016-01-21 20:10:00,2016-01-21 23:00:00,ATA,TU 320IMU,-8.0,10200.0,20,21,...,5.2,0.0,10.8,10.3,0.0,5.0,Turkey,Istanbul,0,0
4,CMN,TUN,2016-01-22 17:45:00,2016-01-22 20:10:00,ATA,TU 320IMR,-37.0,8700.0,17,22,...,16.0,0.0,9.8,10.6,2.0,4.6,Morocco,Casablanca,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99742,TUN,DJE,2018-04-18 08:20:00,2018-04-18 09:10:00,ATA,TU 31BIMQ,8.0,3000.0,8,18,...,16.8,0.0,14.2,17.1,0.0,20.2,Tunisia,Tunis,0,0
99743,ORY,DJE,2018-12-05 10:15:00,2018-12-05 13:05:00,ATA,TU 736IOL,20.0,10200.0,10,5,...,9.6,1.2,10.7,17.3,0.0,12.5,France,Paris,0,0
99744,BRU,DJE,2018-12-05 09:45:00,2018-12-05 12:50:00,ATA,TU 736IOR,15.0,11100.0,9,5,...,19.1,0.0,12.4,17.3,0.0,12.5,Belgium,Brussels,0,0
99745,ORY,TUN,2018-12-04 18:30:00,2018-12-04 20:55:00,ATA,TU 32AIMI,22.0,8700.0,18,4,...,10.2,1.2,8.2,16.7,0.0,19.6,France,Paris,0,0


In [10]:
y = df['target']
X = df.drop(columns=['target'])

In [13]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
# Define the categorical features
num_col = ['duration','dep_temp', 'dep_precip', 'dep_wind', 'arr_temp',
       'arr_precip', 'arr_wind', 'holiday_length']
cat_col = ['departure_point', 'arrival_point', 'flight_status', 'aircraft_code','dep_hour',
       'dep_day', 'dep_month', 'dep_dayofweek', 'dep_quarter', 'dep_season',
       'dep_is_weekend', 'dep_time_of_day', 'arr_hour', 'arr_day', 'arr_month',
       'arr_dayofweek', 'arr_quarter', 'arr_season', 'arr_is_weekend',
       'arr_time_of_day', 'route', 'is_holiday', 'Country', 'City']



# Use sparse output for OneHotEncoder to save memory
encoder = OneHotEncoder(sparse=True, handle_unknown='ignore')
scaler = StandardScaler()

# Fit and transform categorical columns (sparse matrix)
X_cat_sparse = encoder.fit_transform(X[cat_col])

# Scale only the numerical columns and convert to float32
X_num_scaled = scaler.fit_transform(X[num_col]).astype(np.float32)

# Convert sparse matrix to float32 and combine with numerical features
from scipy import sparse
X_encoded_scaled = sparse.hstack([X_num_scaled, X_cat_sparse.astype(np.float32)]).tocsr()

# Split the encoded and scaled data
X_train_1, X_train_2, y_train_1, y_train_2 = train_test_split(
    X_encoded_scaled, y, stratify=y, test_size=0.2, random_state=RSEED
)

c:\Users\Mahesh\Spiced\water_ml_ops\ds-ml-project\.venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [14]:
# 2. Train-test split
from sklearn.ensemble import GradientBoostingRegressor, VotingRegressor
from sklearn.neighbors import KNeighborsRegressor


# Create the column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', num_col),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_col)
    ])

lr = LinearRegression()
rf = RandomForestRegressor(random_state=42)
gb = GradientBoostingRegressor(random_state=42)
knn = KNeighborsRegressor()
xgb = XGBRegressor(random_state=42)
ridge = Ridge(alpha=1.0, solver='lsqr')
# 4. Define regressors
regressors = [
    # ("lr", LinearRegression()),
    #("rf", RandomForestRegressor(n_estimators=100, random_state=42))
    #  ("gb", GradientBoostingRegressor(n_estimators=100, random_state=42)),
    #  ("ridge", ridge),
    # ("knn", knn),
    ("xgb", xgb)
]

# 5. Voting Regressor
voting_reg = VotingRegressor(estimators=regressors)

# 6. Final pipeline
pipeline = Pipeline(steps=[("preprocessor", preprocessor),
                           ("regressor", xgb)])


# # 8. Train model
pipeline.fit(X_train_1, y_train_1)

# # 9. Predict
y_pred = pipeline.predict(X_train_2)


ValueError: Specifying the columns using strings is only supported for pandas DataFrames

In [ ]:
pipeline.fit(X_train, y_train)
# Predict and evaluate
y_pred = pipeline.predict(X_test)


In [ ]:
y_pred[y_pred < 0] = 0
y_test = y_test.clip(lower=0)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Test RMSE: {rmse:.2f}")

In [ ]:
param_distributions = {
    "regressor__xgb__subsample": [0.8, 1.0],
    "regressor__xgb__n_estimators": [100],
    "regressor__xgb__max_depth": [3, 4],

   "regressor__rf__n_estimators": [100, 200, 300],
    "regressor__rf__max_depth": [None, 10, 20],
    "regressor__rf__min_samples_split": [2, 5, 10],
    "regressor__rf__min_samples_leaf": [1, 2, 4],
    "regressor__rf__bootstrap": [True, False],
}

random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=20,
    cv=5,
    scoring='neg_mean_squared_error',
    verbose=1,
    n_jobs=-1,
    random_state=42)

random_search.fit(X_train, y_train)

In [ ]:
best_pipeline = random_search.best_estimator_

# Predict and evaluate
y_pred = best_pipeline.predict(X_test)

y_pred[y_pred < 0] = 0
y_test = y_test.clip(lower=0)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Best Params: {random_search.best_params_}")
print(f"Test RMSE: {rmse:.2f}")

In [ ]:
# Define the categorical features
num_cols = ['duration']
cat_cols = ['departure_point', 'arrival_point', 'flight_status', 'aircraft_code']


# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', num_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
    ]
)

# Define individual regressors
rf = RandomForestRegressor(n_estimators=100, random_state=42)
xgb = XGBRegressor(objective='reg:squarederror', random_state=42)
ridge = Ridge(alpha=1.0, solver='lsqr')
knn = KNeighborsRegressor(n_neighbors=5)

# Voting Regressor
voting_regressor = VotingRegressor(
    estimators=[
        # ('rf', rf),
        ('xgb', xgb),
        ('ridge', ridge),
        ('knn', knn)
    ]
)

# Pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', voting_regressor)
])

# Train
pipeline.fit(X_train, y_train)

# Predict
y_pred = pipeline.predict(X_test)

# Evaluate
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"Voting Regressor RMSE: {rmse:.2f}")


In [ ]:
# Define individual models
ridge = Ridge(solver='lsqr')
gb = GradientBoostingRegressor(random_state=42)
xgb = XGBRegressor(objective='reg:squarederror', random_state=42)

# Create the voting regressor
voting = VotingRegressor(estimators=[
    # ('ridge', ridge),
    # ('gb', gb),
    ('xgb', xgb)
])

# Final pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', voting)
])

# Hyperparameter space
# param_distributions = {
#     # 'regressor__ridge__alpha': [0.1, 1.0, 10.0],
#     # 'regressor__gb__n_estimators': [100, 200],
#     # 'regressor__gb__max_depth': [3, 5, 7],
#     # 'regressor__gb__learning_rate': [0.01, 0.1, 0.2],
#     'regressor__xgb__n_estimators': [100, 200],
#     'regressor__xgb__max_depth': [3, 5, 7],
#     'regressor__xgb__learning_rate': [0.01, 0.1, 0.2],
#     'regressor__xgb__subsample': [0.8, 1.0]
# }

param_distributions = {
    'regressor__xgb__n_estimators': [100, 200, 300],
    'regressor__xgb__learning_rate': [0.01, 0.05, 0.1],
    'regressor__xgb__max_depth': [3, 5, 6, 8],
    'regressor__xgb__min_child_weight': [1, 3, 5],
    'regressor__xgb__gamma': [0, 0.1, 0.2],
    'regressor__xgb__subsample': [0.7, 0.8, 1.0],
    'regressor__xgb__colsample_bytree': [0.7, 0.8, 1.0],
    'regressor__xgb__reg_alpha': [0, 0.1, 0.5],
    'regressor__xgb__reg_lambda': [1.0, 1.5, 2.0]
}

# Randomized Search
search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=20,
    scoring='neg_root_mean_squared_error',
    cv=3,
    verbose=2,
    n_jobs=-1,
    random_state=42
)

# Fit model
search.fit(X_train, y_train)

# Best model and evaluation
print("Best parameters:\n", search.best_params_)
y_pred = search.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Test RMSE: {rmse:.2f}")
